In [130]:
from IPython.core.interactiveshell import InteractiveShell   #默认打印所有的输出
InteractiveShell.ast_node_interactivity = "all"

In [131]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度。**]
重要的是，我们不会在每次对一个参数求导时都分配新的内存。
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。


In [132]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad == None  # 默认值是None，此时还没有对x进行求导

tensor([0., 1., 2., 3.], requires_grad=True)

True

requires_grad=True 的作用是让 backward 可以追踪这个参数x并且计算它的梯度。并用x.grad来访问;

In [133]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

grad_fn用来记录变量是怎么来的，方便计算梯度，y = 2 * torch.dot(x, x),grad_fn记录了y由x计算的过程。

In [134]:
y.backward()   # 通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度
x.grad         # 打印这些梯度

tensor([ 0.,  4.,  8., 12.])

函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于$\mathbf{x}$的梯度应为$4\mathbf{x}$。
让我们快速验证这个梯度是否计算正确。

In [135]:
x.grad == 4 * x    

tensor([True, True, True, True])

In [136]:
# 在默认情况下，PyTorch会累积梯度，每次对关于x的函数y进行一次backward，计算出的梯度都会加到x.gard上；因此我们需要清除之前的值
x.grad.zero_()      # 让x的梯度清零
y = x.sum()
y.backward()
x.grad

tensor([0., 0., 0., 0.])

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播  
以上讲的是标量y对向量x进行求导，当y不是标量，而是向量或者更高阶的张量时，对向量x或者更高阶的张量x求导时，又是什么样的情况呢？  
当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。 对于高阶和高维的y和x，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中）， 但当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [137]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x
x.grad.zero_()
y = x * x        # x * x 是矩阵乘，结果y是一个矩阵
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()        # 对于高阶张量求导，通常是，先求和成张量，即y.sum()，然后再backward
x.grad

tensor([0., 1., 2., 3.], requires_grad=True)

tensor([0., 0., 0., 0.])

tensor([0., 2., 4., 6.])

## 分离计算
有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数  
这里可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息。 换句话说，梯度不会向后流经u到x。 因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。

In [138]:
x.grad.zero_()
y = x * x        # y是关于x的函数
u = y.detach()    # u 只是数值上等于 y 
z = u * x         # u是一个常数，而y是x的函数

z.sum().backward()
x.grad == u

tensor([0., 0., 0., 0.])

tensor([True, True, True, True])

由于记录了`y`的计算结果，我们可以随后在`y`上调用反向传播，
得到`y=x*x`关于的`x`的导数，即`2*x`。


In [139]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([0., 0., 0., 0.])

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：
[**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**]。
在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [140]:
def f(a):                 #  通过 python控制流 求自变量a的函数f(a)
    b = a * 2
    while b.norm() < 1000:     # norm是求范数
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [142]:
a = torch.randn(size=(), requires_grad=True)   # size=() 表示 a是一个标量， randn 表示 从均值为0、标准差为1的标准高斯分布（正态分布）中随机采样。
d = f(a)
d.backward()

我们现在可以分析上面定义的f函数。 请注意，它在其输入a中是分段线性的。 换言之，对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输入a，因此可以用d/a验证梯度是否正确。 (不是很懂)

In [143]:
a.grad == d / a   

tensor(True)